## Adicionando filtros Interativos

In [2]:
df = pd.read_csv('../data/kc_house_data.csv')

df['is_waterfront'] =df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# Define level
df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else
                                          1 if (x >= 321950) & (x < 450000) else
                                          2 if (x >= 450000) & (x < 645000) else
                                          3
                                    )
df['level'] = df['level'].astype( int )

In [3]:
# Interative buttons
price_limit = widgets.IntSlider(value = 540000,
                                min = 75000,
                                max = 7700000,
                                step = 1,
                                description = 'Maximum price',
                                disable = False,
                                )

waterfront_bar = widgets.Dropdown(options = df['is_waterfront'].unique().tolist(),
                                  value = 'yes',
                                  description = 'Water View',
                                  disable = False                                    
                                  )

In [4]:
def update_map(df, waterfront_bar, price_limit):
    houses = df[(df['price'] <= price_limit) & (df['is_waterfront'] == waterfront_bar)][['id', 'lat', 'long', 'price', 'level']]

    fig = px.scatter_mapbox( houses,
                            lat = 'lat',
                            lon = 'long',
                            color = 'level',
                            size = 'price',
                            color_continuous_scale = px.colors.cyclical.IceFire,
                            size_max = 15,
                            zoom = 10 )

    fig.update_layout( mapbox_style = 'open-street-map' )
    fig.update_layout( height = 800, margin = {'r':0, 't':0, 'l':0, 'b':0})
    fig.show()

In [5]:
widgets.interactive(update_map, df = fixed( df ), waterfront_bar = waterfront_bar, price_limit = price_limit)

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

## Interatividade com o Dashboard

In [6]:
df = pd.read_csv('../data/kc_house_data.csv')

# Change date format
df['year'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y' )
df['date'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%m-%d' )
df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U' )

# Widgets para controlar os dados
date_limit = widgets.SelectionSlider(options = df['date'].sort_values().unique().tolist(),
                                     value = '2014-12-01',
                                     description = 'Disponível',
                                     continuous_update = False,
                                     orientation = 'horizontal',
                                     readout = True
                                     )

def update_map( df, limit ):
    # Filtrar dados
    df = df[df['date'] <= limit].copy()
    
    fig = plt.figure(figsize = (21,12) ) 
    specs = gridspec.GridSpec( ncols = 2, nrows = 2, figure = fig )
    
    ax1 = fig.add_subplot( specs[ 0, : ] ) # Primeira linha
    ax2 = fig.add_subplot( specs[ 1, 0 ] ) # Segunda linha e primeira coluna
    ax3 = fig.add_subplot( specs[ 1, 1 ] ) # Segunda linha e segunda coluna
    
    by_year = df[['id','year']].groupby('year').sum().reset_index()
    sns.barplot(x = 'year', y = 'id', data = by_year, ax = ax1 )
    
    by_day = df[['id','date']].groupby('date').mean().reset_index()
    sns.lineplot( x = 'date', y = 'id', data = by_day, ax = ax2 )
    ax2.set_title( 'AVG Price by Day')
    
    by_week_of_year = df[['id','year_week']].groupby('year_week').mean().reset_index()
    sns.barplot( x = 'year_week',y = 'id', data = by_week_of_year, ax  = ax3 )
    ax3.set_title( 'AVG Price by Week of Year')
    plt.xticks( rotation = 60 );

In [7]:
widgets.interactive( update_map, df = fixed(df),
                     limit          = date_limit,
                   )

interactive(children=(SelectionSlider(continuous_update=False, description='Disponível', index=212, options=('…

# Perguntas do CEO

1. Qual a média do preço de compra dos imóveis por nível?
    - Nível 0: preço entre  0.00 e  321.950 dólares
    - Nível 1: preço entre  321.950 e  450.000 dólares
    - Nível 2: preço entre  450.000 e  645.000 dólares
    - Nível 3: Preço acima de  645.000 dólares
    

2. Qual a média do tamanho da sala de estar dos imóveis por Size?
    - Size 0 -> Tamanho entre 0 e 1427 sqft
    - Size 1 -> Tamanho entre 1427 e 1910 sqft
    - Size 2 -> Tamanho entre 1910 e 2550 sqft
    - Size 3 -> Tamanho acima de 2550 sqft
    
    
3. Adicione as seguintes informações ao conjunto de dados original:
    - Place ID: Identificação da localização
    - OSM Type: Open Street Map type
    - Country: Nome do país
    - Country Code: Código do País


4. Adicione os seguintes filtros no Mapa:
    - Tamanho mínimo da área da sala de estar
    - Número mínimo de banheiros
    - Valor máximo do preço
    - Tamanho máximo da área do porão
    - Filtro das Condições do imóvel
    - Filtro por ano de construção
    
    
5. Adicione os seguintes filtros no Dashboard:
    - Filtro por data disponível para compra
    - Filtro por ano de renovação
    - Filtro se possui vista para água ou não

In [8]:
df = pd.read_csv('../data/kc_house_data.csv')

## 1. Qual a média do preço de compra dos imóveis por nível?
    - Nível 0: preço entre  0.00 e  321.950 dólares
    - Nível 1: preço entre  321.950 e  450.000 dólares
    - Nível 2: preço entre  450.000 e  645.000 dólares
    - Nível 3: Preço acima de  645.000 dólares

In [9]:
df['level'] = df['price'].apply(lambda x: 0 if x  <  321950.0 else
                            1 if (x >= 321950.0 and x < 450000.0) else
                            2 if (x >= 450000.0 and x < 645000.0) else 3 )
df[['level','price']].groupby('level').mean()                              

,price
level,
0,251544.62
1,383585.21
2,536164.42
3,985008.98


## 2. Qual a média do tamanho da sala de estar dos imóveis por Size?
    - Size 0 -> Tamanho entre 0 e 1427 sqft
    - Size 1 -> Tamanho entre 1427 e 1910 sqft
    - Size 2 -> Tamanho entre 1910 e 2550 sqft
    - Size 3 -> Tamanho acima de 2550 sqft

In [10]:
df['living_type'] = df['sqft_living'].apply(lambda x: 0 if x  <  1427 else
                            1 if (x >= 1427 and x < 1910) else
                            2 if (x >= 1910 and x < 2550) else 3 )
df[['living_type','sqft_living']].groupby('living_type').mean()     

,sqft_living
living_type,
0,1123.78
1,1661.20
2,2202.70
3,3318.99


## 3. Adicione as seguintes informações ao conjunto de dados original:
    - Place ID: Identificação da localização
    - OSM Type: Open Street Map type
    - Country: Nome do país
    - Country Code: Código do País
    
#### Utilizando processamento paralelo 

In [11]:
aux = df.loc[:,['id','lat','long']].sample(30)
aux['lat+long'] = aux['lat'].astype( str ) + "," + aux['long'].astype( str )
aux = aux[['id', 'lat+long']]

In [12]:
# Iniciar Nominatim API
#geolocator = Nominatim(user_agent = 'geoapiExercises', timeout = None)
#import time
#from geopy.geocoders import Nominatim

#def get_data( x ):
#    index, row = x  # Necessário pois o df foi transformado em iterrows()
#    time.sleep( 1 ) # Necessário para evitar loop infinito
    
#    # Chamada API
#    response = geolocator.reverse(x).raw
#    address = response['address']
    
#    place_id = response['place_id']        if 'place_id'     in response else "NA"
#    osm_type = response['osm_type']        if 'osm_type'     in response else "NA"
#    country = address['country']           if 'country'      in address  else "NA"
#    country_code = address['country_code'] if 'country_code' in address  else "NA"
    
#    return place_id, osm_type, country, country_code

In [13]:
import defs

p = Pool ( 3 )

start = time.process_time()
aux[['place_id', 'osm_type', 'country', 'country_code']] = p.map( defs.get_data, aux.iterrows() )
end = time.process_time()

print("Time Elapsed: ", end - start)

Time Elapsed:  0.027853145999999995


## 4. Adicione os seguintes filtros no Mapa:
    - Tamanho mínimo da área da sala de estar
    - Número mínimo de banheiros
    - Valor máximo do preço
    - Tamanho máximo da área do porão
    - Filtro das Condições do imóvel
    - Filtro por ano de construção

In [14]:
min_living = widgets.SelectionSlider( options = df['sqft_living'].sort_values().unique().tolist(),
                                      value = df['sqft_living'].min(),
                                      disabled = False,
                                      orientation = 'horizontal',
                                      continuous_update = False,
                                      readout = True,
                                      description = 'Living Room Size')

min_bathrooms = widgets.SelectionSlider( options = df['bathrooms'].sort_values().unique().tolist(),
                                         value = df['bathrooms'].min(),
                                         disabled = False,
                                         orientation = 'horizontal',
                                         continuous_update = False,
                                         readout = True)

max_price = widgets.IntSlider(  value = int( df['price'].mean() ),
                                min   = int( df['price'].min() ),
                                max   = int( df['price'].max() ),
                                step  = 1,
                                description = 'Max Price',
                                )

max_basement = widgets.SelectionSlider(  options = df['sqft_basement'].sort_values().unique().tolist(),
                                         value = df['sqft_basement'].max(),
                                         disabled = False,
                                         orientation = 'horizontal',
                                         continuous_update = False,
                                         readout = True,
                                         description = 'Basement Size')

condition = widgets.Dropdown( options = df['condition'].sort_values().unique().tolist(),
                              value = df['condition'].min(),
                              description = 'House Condition',
                              disable = False )

yr_built = widgets.SelectionSlider( options = df['yr_built'].sort_values().unique().tolist(),
                              value = df['yr_built'].min(),
                              disabled = False,
                              orientation = 'horizontal',
                              continuous_update = False,
                              readout = True,    
                              description = 'Year Built')


def update_map(data, living_filter, bathrooms_filter, basement_filter, price_filter, condition_filter, yr_built_filter):
    max_level = data['level'].max()
    min_level = data['level'].min()
    df = data[['lat', 'long', 'price', 'level']][ (data['sqft_living'  ] >= living_filter   )  &
                                                  (data['bathrooms'    ] >= bathrooms_filter)  &
                                                  (data['sqft_basement'] <= basement_filter )  &
                                                  (data['price'        ] <= price_filter    )  &
                                                  (data['condition'    ] == condition_filter)  &
                                                  (data['yr_built'     ] >= yr_built_filter )  &
                                                  (data['bathrooms'    ] >= bathrooms_filter)]

    fig = px.scatter_mapbox( data_frame = df,
                             lat = 'lat',
                             lon = 'long',
                             color = 'level',
                             range_color = [min_level, max_level],
                             size = 'price',
                             color_continuous_scale = 'deep',# px.colors.cyclical.IceFire,
                             size_max = 15,
                             zoom = 10 )
    fig.update_layout( mapbox_style = 'open-street-map')
    fig.update_layout( height = 600, margin = {'r':0,'t':0,'l':0,'b':0})
    fig.show()

In [15]:
widgets.interactive( update_map, data = fixed(df),
                     living_filter    = min_living,
                     bathrooms_filter = min_bathrooms,
                     basement_filter  = max_basement,
                     price_filter     = max_price,
                     condition_filter = condition,
                     yr_built_filter  = yr_built       )

interactive(children=(SelectionSlider(continuous_update=False, description='Living Room Size', options=(290, 3…

## 5. Adicione os seguintes filtros no Dashboard:
    - Filtro por data disponível para compra
    - Filtro por ano de renovação
    - Filtro se possui vista para água ou não

In [16]:
df = pd.read_csv('../data/kc_house_data.csv')
df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# Change date format
df['year'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y' )
df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U' )
df['date'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%m-%d' )
df['is_waterfront'] = df['waterfront'].apply(lambda x: "No" if x == 0 else "Yes")

date_filter = widgets.SelectionSlider(options = df['date'].sort_values().unique().tolist(),
                                      value = '2014-12-01',
                                      orientation = 'horizontal',
                                      description = "Min Date",
                                      continuous_update = False,
                                      readout = True)
yr_renovated = widgets.SelectionSlider(options = df['yr_renovated'].sort_values().unique().tolist(),
                                      value = df['yr_renovated'].min(),
                                      disabled = False,
                                      orientation = 'horizontal',
                                      description = "Yr Renovated",
                                      continuous_update = False,
                                      readout = True)
is_waterfront = widgets.Dropdown(options = df['is_waterfront'].unique().tolist(),   
                                 disabled = False,
                                 description = "Waterfront"                             
                                )


# Widgets para controlar os dados


def update_map( df, date_filter, yr_renovated, is_waterfront):
    # Filtrar dados
    df = df[(df['date'] <= date_filter) &
            (df['yr_renovated'] >= yr_renovated) &
            (df['is_waterfront'] == is_waterfront)].copy()
    
    fig = plt.figure(figsize = (21,12) ) 
    specs = gridspec.GridSpec( ncols = 2, nrows = 2, figure = fig )
    
    ax1 = fig.add_subplot( specs[ 0, : ] ) # Primeira linha
    ax2 = fig.add_subplot( specs[ 1, 0 ] ) # Segunda linha e primeira coluna
    ax3 = fig.add_subplot( specs[ 1, 1 ] ) # Segunda linha e segunda coluna
    
    by_year = df[['id','year']].groupby('year').sum().reset_index()
    sns.barplot(x = 'year', y = 'id', data = by_year, ax = ax1 )
    
    by_day = df[['id','date']].groupby('date').mean().reset_index()
    sns.lineplot( x = 'date', y = 'id', data = by_day, ax = ax2 )
    ax2.set_title( 'AVG Price by Day')
    
    by_week_of_year = df[['id','year_week']].groupby('year_week').mean().reset_index()
    sns.barplot( x = 'year_week',y = 'id', data = by_week_of_year, ax  = ax3 )
    ax3.set_title( 'AVG Price by Week of Year')
    plt.xticks( rotation = 60 );

In [17]:
widgets.interactive(update_map, df = fixed( df ),
                    date_filter = date_filter,
                    yr_renovated = yr_renovated,
                    is_waterfront = is_waterfront)

interactive(children=(SelectionSlider(continuous_update=False, description='Min Date', index=212, options=('20…